# [Module 2.2] 세이지 메이커 인퍼런스

본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 배포 준비
- 3. 엔드포인트 생성
- 4. 엔드 포인트 추론
- 5. 로컬 엔드 포인트 삭제


---    
    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.

In [2]:
%store -r artifact_path
%store -r bucket
%store -r prefix

# 2. 배포 준비

In [3]:
import sagemaker

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 2.1. 모델 아티펙트 확인

In [4]:
print("artifact_path: ", artifact_path)

artifact_path:  s3://sagemaker-us-east-1-057716757052/pytorch-training-2023-10-28-07-43-01-333/output/model.tar.gz


## 2.2. 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [5]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

## 2.3. 추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [6]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of batch_size:  256


## 2.4. 추론할 Paylaod 하나를 생성

In [7]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


payload:  {'user': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'item': [25, 1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]}


# 3. 엔드포인트 생성
- 이 과정은 세이지 메이커 엔드포인트를 생성합니다.
- 7분 정도 소요 됩니다.



In [8]:
import os
import time
from sagemaker.pytorch.model import PyTorchModel

# instance_type = 'ml.p2.xlarge' # $ 1.831 per Hour
instance_type = 'ml.g4dn.xlarge' # $ 0.906 per Hour

In [9]:
%%time 

endpoint_name = "sm-ncf-{}".format(int(time.time()))


sm_pytorch_model = PyTorchModel(model_data=artifact_path,
                                   role=role,
                                   entry_point='inference.py',
                                   source_dir = 'src',
                                   framework_version='2.0.1',
                                   py_version='py310',
                                   model_server_workers=1,
                                  )

sm_predictor = sm_pytorch_model.deploy(instance_type= instance_type, 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True,
                        )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
--------!CPU times: user 629 ms, sys: 44.2 ms, total: 673 ms
Wall time: 4min 34s


# 4. 엔드 포인트 추론

## 4.1. 프리딕터에 JSONSerializer 로 초기화
- payload 로 JSON type 으로 제공하면 JSONSerializer 가 String 으로 직렬화하여 제공 함.

In [10]:
from sagemaker.serializers import JSONSerializer
sm_predictor.serializer = JSONSerializer('application/json')

In [11]:
sm_predictor.predict(payload)

array([ 128,   25,  174,  273,  464, 1902,   58,  175, 1064,  617])

## 4.2. Boto3 invoke_endpoint() 로 추론

In [12]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')



### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [13]:
import json
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

--- 0.07393670082092285 seconds ---
result:  ['[128, 25, 174, 273, 464, 1902, 58, 175, 1064, 617]']


# 5. 클라우드 엔드 포인트 삭제
- 기존에 생성한 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 삭제

In [14]:
from inference_utils import delete_endpoint

# client = sagemaker.local.LocalSagemakerClient()
client = boto3.Session().client('sagemaker')
delete_endpoint(client, endpoint_name)

#### Start
--- Deleted model: pytorch-inference-2023-10-28-10-49-54-489
--- Deleted endpoint: sm-ncf-1698490192
--- Deleted endpoint_config: sm-ncf-1698490192
